In [ ]:
!pip install exetera

In [ ]:
%matplotlib inline
from datetime import datetime, timedelta
import numpy as np
import h5py
import matplotlib.pyplot as plt

from exetera.core.session import Session

start_date = # datetime(year, month, date), inclusive
end_date = # datetime(year, month, date), exclusive
filename = # file name for source dataset

dates = [end_date]
cur_date = end_date - timedelta(days=7)
while cur_date >= start_date:
    dates.append(cur_date)
    cur_date -= timedelta(days=7)
dates.reverse()

In [ ]:

with h5py.File(filename, 'r') as src:
    s = Session()
    
    t_src = src['tests']
    test_created_at_ = s.get(t_src['date_taken_specific']).data[:]
    test_results_ = s.get(t_src['result']).data[:]
    print(np.unique(test_results_, return_counts=True))
    partitions = len(dates)-1
    negative_tests_by_range = np.zeros(partitions, dtype=np.int32)
    positive_tests_by_range = np.zeros(partitions, dtype=np.int32)

    for d in range(partitions):
        start_timestamp = dates[d].timestamp()
        end_timestamp = dates[d+1].timestamp()
        in_current_month = (start_timestamp <= test_created_at_) & (test_created_at_ < end_timestamp)
        print(dates[d], in_current_month.sum())
        
        monthly_tests = s.apply_filter(in_current_month, test_results_)
        
        result_counts = np.unique(monthly_tests, return_counts=True)
        result_counts = dict(zip(result_counts[0], result_counts[1]))
        print(result_counts)
        negative_tests_by_range[d] = result_counts[3]
        positive_tests_by_range[d] = result_counts[4]


In [ ]:
width = 1
widths = [width * d for d in range(partitions)]

fig, ax = plt.subplots(2, 1, figsize=(10, 10))

negtests = ax[0].bar(widths, negative_tests_by_range)
postests = ax[0].bar(widths, positive_tests_by_range, bottom=negative_tests_by_range)

ax[0].set_title("COVID-19 test results, by week")
ax[0].set_xticks(np.arange(partitions))
ax[0].set_xticklabels([datetime.strftime(d, '%Y-%m-%d') for d in dates[:-1]], rotation=270)
ax[0].legend((negtests, postests), ("Negative tests", "Positive tests"))
ax[0].set_xlabel("Week starting")
ax[0].set_ylabel("Tests per week")

pos_fraction = positive_tests_by_range / (positive_tests_by_range + negative_tests_by_range)
pfbar = ax[1].bar(widths, pos_fraction, color="#ff7f0e")

ax[1].set_title("COVID-19 positive tests by fraction of test results, by week")
ax[1].set_xticks(np.arange(partitions))
ax[1].set_xticklabels([datetime.strftime(d, '%Y-%m-%d') for d in dates[:-1]], rotation=270)
ax[1].legend((pfbar,), ("Positive test fraction",))
ax[1].set_xlabel("Week starting")
ax[1].set_ylabel("Positive test fraction")

fig.tight_layout(h_pad=2.5)
fig.show()
